# Calculate overshooting frequencies at each grid point for the obs climo
#### Get the frequencies for Tb <= local cpT ~~Tb <= threshold and Tb <= cpT~~
First need to get the concatenated Tb files (using cdo) from `cat_Tb_cpT_time.sh` ~~`cat_Tb_time.sh`~~


In [1]:
import dask
import numpy as np
import xarray as xr

from dask.diagnostics import ProgressBar


In [2]:
file_path = "/work/bb1153/b380887/big_obs_climo/"
scratch_path = "/scratch/b/b380887/"


In [3]:
season = "JJA"
years_str = "2007-2010"


In [4]:
if season == "DJF":
    region_list = ["AMZ", "SPC1", "SPC2", "ECP", "IOS"] 

elif season == "JJA":
    region_list = ["AFR", "WPC", "ECP", "IOE"]


### Get the Tb and reindexed (local) cpT files

In [5]:
tb_dict = {}
cpT_dict = {}

for region in region_list:
    cpT = xr.open_dataset(
        scratch_path + "ERA5_cpT_reindexed_{s}{y}_{r}.nc".format(s=season, y=years_str, r=region),
    )["t"]
    tb_dict[region] = xr.open_dataset(
        scratch_path + "MERGIR_Tb_4km_{s}{y}_{r}.nc4".format(s=season, y=years_str, r=region),
    )["Tb"]
    cpT_dict[region] = cpT.reindex({"time": tb_dict[region].time}, method="nearest") # get from 1 hour --> 30 min



#### Get the frequencies

In [6]:
chunks = {"time": 4} # this is faster than "auto"


In [7]:
%%time
# os_freqs_dict = {}

# to see how the results change w/ different threshold
os_freqs_dict_m2 = {} # Tb < (Tcp - 2)
os_freqs_dict_m4 = {} # Tb < (Tcp - 4)

for region in region_list:
    tb_chunked = tb_dict[region].chunk(chunks)
    cpT_chunked = cpT_dict[region].chunk(chunks)
    # counts_chunked = tb_chunked.where(tb_chunked < cpT_chunked).count(dim="time")
    counts_chunked_m2 = tb_chunked.where(tb_chunked < (cpT_chunked - 2)).count(dim="time")
    counts_chunked_m4 = tb_chunked.where(tb_chunked < (cpT_chunked - 4)).count(dim="time")

    with ProgressBar():
        # counts = counts_chunked.compute()
        counts_m2 = counts_chunked_m2.compute()
    with ProgressBar():
        counts_m4 = counts_chunked_m4.compute()
        
    # os_freqs_dict[region] = counts / len(tb_dict[region].time)
    os_freqs_dict_m2[region] = counts_m2 / len(tb_dict[region].time)
    os_freqs_dict_m4[region] = counts_m4 / len(tb_dict[region].time)

    print(region, "done")
    

[########################################] | 100% Completed |  5min 39.2s
[########################################] | 100% Completed |  2min 43.2s
AFR done
[########################################] | 100% Completed |  5min 19.1s
[########################################] | 100% Completed |  3min  2.9s
WPC done
[########################################] | 100% Completed |  5min 43.8s
[########################################] | 100% Completed |  5min 13.4s
ECP done
[########################################] | 100% Completed |  5min 54.1s
[########################################] | 100% Completed |  4min 34.8s
IOE done
CPU times: user 21min 8s, sys: 5min 45s, total: 26min 54s
Wall time: 38min 12s


In [8]:
for region in region_list:
    if region == "SPC1":
        # do both SPC regions for "SPC1"
        # os_freqs_1 = os_freqs_dict["SPC1"]
        # os_freqs_2 = os_freqs_dict["SPC2"]
        # os_freqs = xr.concat([os_freqs_1, os_freqs_2], dim="lon")
        os_freqs_m2_1 = os_freqs_dict_m2["SPC1"]
        os_freqs_m2_2 = os_freqs_dict_m2["SPC2"]
        os_freqs_m2 = xr.concat([os_freqs_m2_1, os_freqs_m2_2], dim="lon")
        os_freqs_m4_1 = os_freqs_dict_m4["SPC1"]
        os_freqs_m4_2 = os_freqs_dict_m4["SPC2"]
        os_freqs_m4 = xr.concat([os_freqs_m4_1, os_freqs_m4_2], dim="lon")
        save_region = "SPC"
    elif region == "SPC2":
        # skip - already done
        continue
    else:
        # os_freqs = os_freqs_dict[region]
        os_freqs_m2 = os_freqs_dict_m2[region]
        os_freqs_m4 = os_freqs_dict_m4[region]
        save_region = region

    # os_freqs.attrs = {
    #     "threshold_variable": "brightness temperature",
    #     "threshold_description": "Tb < collocated 0.25deg cold point (not time mean)",
    #     "units": "fraction (not percentage)"
    # }
    os_freqs_m2.attrs = {
        "threshold_variable": "brightness temperature",
        "threshold_description": "Tb < collocated 0.25deg cold point - 2K (not time mean)",
        "units": "fraction (not percentage)"
    }
    os_freqs_m4.attrs = {
        "threshold_variable": "brightness temperature",
        "threshold_description": "Tb < collocated 0.25deg cold point - 4K (not time mean)",
        "units": "fraction (not percentage)"
    }

    # ds_os = xr.Dataset({"os_freqs": os_freqs})
    ds_os_m2 = xr.Dataset({"os_freqs": os_freqs_m2})
    ds_os_m4 = xr.Dataset({"os_freqs": os_freqs_m4})

    if region == "SPC2":
        print("got here")
    # out_path = file_path + "os_frequencies/os_freq_Tb_below_cpT_{s}{y}_{r}.nc".format(s=season, y=years_str, r=save_region)
    # ds_os.to_netcdf(out_path)
    out_path_m2 = file_path + "os_frequencies/os_freq_Tb_below_cpT-2K_{s}{y}_{r}.nc".format(s=season, y=years_str, r=save_region)
    ds_os_m2.to_netcdf(out_path_m2)
    out_path_m4 = file_path + "os_frequencies/os_freq_Tb_below_cpT-4K_{s}{y}_{r}.nc".format(s=season, y=years_str, r=save_region)
    ds_os_m4.to_netcdf(out_path_m4)
    

In [17]:
if "SPC1" in region_list:
    for region in ["SPC1", "SPC2"]:
        # os_freqs = os_freqs_dict[region]
        os_freqs_m2 = os_freqs_dict_m2[region]
        os_freqs_m4 = os_freqs_dict_m4[region]
        save_region = region

#         os_freqs.attrs = {
#             "threshold_variable": "brightness temperature",
#             "threshold_description": "Tb < collocated 0.25deg cold point (not time mean)",
#             "units": "fraction (not percentage)"
#         }
        os_freqs_m2.attrs = {
            "threshold_variable": "brightness temperature",
            "threshold_description": "Tb < collocated 0.25deg cold point - 2K (not time mean)",
            "units": "fraction (not percentage)"
        }
        os_freqs_m4.attrs = {
            "threshold_variable": "brightness temperature",
            "threshold_description": "Tb < collocated 0.25deg cold point - 4K (not time mean)",
            "units": "fraction (not percentage)"
        }

        # ds_os = xr.Dataset({"os_freqs": os_freqs})
        ds_os_m2 = xr.Dataset({"os_freqs": os_freqs_m2})
        ds_os_m4 = xr.Dataset({"os_freqs": os_freqs_m4})
        
        # out_path = file_path + "os_frequencies/os_freq_Tb_below_cpT_{s}{y}_{r}.nc".format(s=season, y=years_str, r=save_region)
        # ds_os.to_netcdf(out_path)
        out_path_m2 = file_path + "os_frequencies/os_freq_Tb_below_cpT-2K_{s}{y}_{r}.nc".format(s=season, y=years_str, r=save_region)
        ds_os_m2.to_netcdf(out_path_m2)
        out_path_m4 = file_path + "os_frequencies/os_freq_Tb_below_cpT-4K_{s}{y}_{r}.nc".format(s=season, y=years_str, r=save_region)
        ds_os_m4.to_netcdf(out_path_m4)
    


## Old way (using time-averaged cpT & calculated threshold)

### Get the threshold files

In [5]:
# thresh_dict = {}
# tb_dict = {}
# cpT_dict = {}
# offset_dict = {}
# chunks = {"time": 4}

# for region in region_list:
#     if region == "SPC1":
#         thresh_025_all = xr.open_dataset(file_path + "Tb_os_thresholds/ERA5_0.25deg_cp_thresh_{s}{y}_SPC.nc".format(y=years_str, s=season))
#         thresh_025 = thresh_025_all.sel(longitude=slice(None, 180))
#     elif region == "SPC2":
#         thresh_025_all = xr.open_dataset(file_path + "Tb_os_thresholds/ERA5_0.25deg_cp_thresh_{s}{y}_SPC.nc".format(y=years_str, s=season))
#         thresh_025 = thresh_025_all.sel(longitude=slice(-180, None))
#     else:
#         thresh_025 = xr.open_dataset(file_path + "Tb_os_thresholds/ERA5_0.25deg_cp_thresh_{s}{y}_{r}.nc".format(r=region, y=years_str, s=season))
    
#     offset_dict[region] = thresh_025.attrs["temp_offset"]
#     tb = xr.open_dataset("/scratch/b/b380887/MERGIR_Tb_4km_{s}{y}_{r}.nc4".format(r=region, y=years_str, s=season))["Tb"]
        
#     thresh_ri = thresh_025["os_threshold"].rename({"longitude": "lon", "latitude": "lat"}).reindex_like(tb, method="nearest")
#     cpT_ri = thresh_025["cpT"].rename({"longitude": "lon", "latitude": "lat"}).reindex_like(tb, method="nearest")
#     thresh_dict[region] = thresh_ri
#     cpT_dict[region] = cpT_ri
#     tb_dict[region] = tb


#### Calculate the frequencies:
Takes ~26 min 

In [6]:
%%time
os_freqs_dict = {}
os_cpT_freqs_dict = {}

for region in region_list:
    tb_chunked = tb_dict[region].chunk({"time": 4})
    counts_chunked = tb_chunked.where(tb_chunked <= thresh_dict[region]).count(dim="time")
    counts_cp_chunked = tb_chunked.where(tb_chunked <= cpT_dict[region]).count(dim="time")
    with ProgressBar():
        counts = counts_chunked.compute()
    with ProgressBar():
        counts_cp = counts_cp_chunked.compute()
    os_freqs_dict[region] = counts / len(tb_dict[region].time)
    os_cpT_freqs_dict[region] = counts_cp / len(tb_dict[region].time)
    print(region, "done")
    

[########################################] | 100% Completed |  5min 45.8s
[########################################] | 100% Completed |  2min 12.7s
AFR done
[########################################] | 100% Completed |  4min 45.3s
[########################################] | 100% Completed | 52.0s
WPC done
[########################################] | 100% Completed |  4min 53.9s
[########################################] | 100% Completed | 53.0s
ECP done
[########################################] | 100% Completed |  4min 15.9s
[########################################] | 100% Completed |  1min 13.3s
IOE done
CPU times: user 12min 51s, sys: 1min 48s, total: 14min 40s
Wall time: 26min 29s


In [6]:
%%time
os_freqs_dict = {}
os_cpT_freqs_dict = {}

# for region in region_list:
for region in ["SPC1", "SPC2"]:
    tb_chunked = tb_dict[region].chunk({"time": 4})
    counts_chunked = tb_chunked.where(tb_chunked <= thresh_dict[region]).count(dim="time")
    counts_cp_chunked = tb_chunked.where(tb_chunked <= cpT_dict[region]).count(dim="time")
    with ProgressBar():
        counts = counts_chunked.compute()
    with ProgressBar():
        counts_cp = counts_cp_chunked.compute()
    os_freqs_dict[region] = counts / len(tb_dict[region].time)
    os_cpT_freqs_dict[region] = counts_cp / len(tb_dict[region].time)
    print(region, "done")
    

[########################################] | 100% Completed |  1min 28.7s
[########################################] | 100% Completed | 14.3s
SPC1 done
[########################################] | 100% Completed |  3min 24.4s
[########################################] | 100% Completed | 35.4s
SPC2 done
CPU times: user 2min 9s, sys: 1min 17s, total: 3min 27s
Wall time: 6min 7s


#### Save all regions as netcdfs:

In [10]:
for region in region_list:
    if region == "SPC1":
        # do both SPC regions for "SPC1"
        os_freqs_1 = os_freqs_dict["SPC1"]
        os_freqs_2 = os_freqs_dict["SPC2"]
        os_freqs = xr.concat([os_freqs_1, os_freqs_2], dim="lon")
        os_cpT_freqs_1 = os_cpT_freqs_dict["SPC1"]
        os_cpT_freqs_2 = os_cpT_freqs_dict["SPC2"]
        os_cpT_freqs = xr.concat([os_cpT_freqs_1, os_cpT_freqs_2], dim="lon")
        save_region = "SPC"
    elif region == "SPC2":
        # skip - already done
        continue
    else:
        os_freqs = os_freqs_dict[region]
        os_cpT_freqs = os_cpT_freqs_dict[region]
        save_region = region

    os_freqs.attrs = {
        "threshold_variable": "brightness temperature",
        "threshold_description": "0.25deg cold point offset by local value",
        "temp_offset": offset_dict[region],
        "units": "fraction (not percentage)"
    }
    
    os_cpT_freqs.attrs = {
        "threshold_variable": "brightness temperature",
        "threshold_description": "Tb < 0.25deg cold point temperature",
        "units": "fraction (not percentage)"
    }
    
    ds_os = xr.Dataset({"os_freqs": os_freqs, "os_cpT_freqs": os_cpT_freqs})

    if region == "SPC2":
        print("got here")
    out_path = file_path + "os_frequencies/os_freq_Tb_{s}{y}_{r}.nc".format(s=season, y=years_str, r=save_region)
    ds_os.to_netcdf(out_path)


#### Also save the SPC regions separately as netcdfs:

In [12]:
if "SPC1" in region_list:
    for region in ["SPC1", "SPC2"]:
        os_freqs = os_freqs_dict[region]
        os_cpT_freqs = os_cpT_freqs_dict[region]

        save_region = region

        os_freqs.attrs = {
            "threshold_variable": "brightness temperature",
            "threshold_description": "0.25deg cold point offset by local value",
            "temp_offset": offset_dict[region],
            "units": "fraction (not percentage)"
        }
        
        os_cpT_freqs.attrs = {
            "threshold_variable": "brightness temperature",
            "threshold_description": "Tb < 0.25deg cold point temperature",
            "units": "fraction (not percentage)"
        }
        
        ds_os = xr.Dataset({"os_freqs": os_freqs, "os_cpT_freqs": os_cpT_freqs})

        out_path = file_path + "os_frequencies/os_freq_Tb_{s}{y}_{r}.nc".format(s=season, y=years_str, r=region)
        ds_os.to_netcdf(out_path)
